In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from os import listdir
from os.path import isfile, join
from datetime import datetime

In [ ]:
measurment_path = '/home/vladimir/BOTAY!/projects/PolarizationNavigator/обработка_результатов/exam_demonstration_results'

In [ ]:
def get_photo_at_exact_time(time: str, arr):
    for i in arr:
        if time in i:
            return i

In [ ]:
def threshhold(arr):
    return (arr > 1) + (arr < 1) * arr

In [ ]:
measurment_file_names = [f for f in listdir(measurment_path) if isfile(join(measurment_path, f))]
dolp_file_names = [f for f in measurment_file_names if 'dolp' in f]
experiment_time = list(set([f[:8] for f in dolp_file_names]))
aop_file_names = [f for f in measurment_file_names if 'aop' in f]
s0_file_names = [f for f in measurment_file_names if 's0' in f]

In [ ]:
print(measurment_file_names)

In [ ]:
experiment_time_sorted = []
for time_string in experiment_time:
    time_obj = datetime.strptime(time_string, "%H_%M_%S")
    experiment_time_sorted.append(time_obj)

experiment_time_sorted = sorted(experiment_time_sorted)
experiment_time_sorted = [datetime.strftime(time_obj, "%H_%M_%S") for time_obj in experiment_time_sorted]

In [ ]:
dolp_file_names_dict = {time: get_photo_at_exact_time(time, dolp_file_names) for time in experiment_time}
aop_file_names_dict = {time: get_photo_at_exact_time(time, aop_file_names) for time in experiment_time}
s0_file_names_dict = {time: get_photo_at_exact_time(time, s0_file_names) for time in experiment_time}

In [ ]:
s0_file_names

In [ ]:
photo_x_shape = np.load(measurment_path + '/' + s0_file_names[0]).shape[0]
photo_y_shape = np.load(measurment_path + '/' + s0_file_names[0]).shape[1]
print(np.load(measurment_path + '/' + s0_file_names[0]).shape)
print(photo_x_shape, photo_y_shape)

In [ ]:
print(len(experiment_time))

In [ ]:
frames_on_plot_range = 1
start = np.arange(20, len(experiment_time) - 2, frames_on_plot_range, dtype=np.int64)

for ind, s in enumerate(start):
    fig, axs = plt.subplots(8, 4, figsize = (14, 18), dpi = 200)
    iterations = np.linspace(s, s + frames_on_plot_range, 8, dtype = np.int64)
    axs[0, 0].set_title("Степень линейной поляризации")
    axs[0, 1].set_title("Угол поляризации")
    axs[0, 3].set_title("Угол поляризации")
    axs[0, 2].set_title("S0")
    for ax, iteration in zip(axs, iterations):
        time = experiment_time_sorted[iteration]


        for a in ax:
            a.set_xticks([])
            a.set_yticks([])
        im = ax[0].imshow(threshhold(np.load(measurment_path + '/' + dolp_file_names_dict[time])), cmap = 'rainbow')
        
        plt.colorbar(im, fraction=0.046, pad=0.04)
        im = ax[1].imshow(np.load(measurment_path + '/' + aop_file_names_dict[time]), cmap = 'rainbow')
        plt.colorbar(im, fraction=0.046, pad=0.04)
        
        ax[2].imshow(np.load(measurment_path + '/' + s0_file_names_dict[time]), cmap = 'gray')
        
        ax[3].imshow(np.load(measurment_path + '/' + s0_file_names_dict[time]), cmap = 'gray', alpha = 1)


        x = np.arange(0, photo_x_shape, 20)
        y = np.arange(0, photo_y_shape, 20)


        X, Y = np.meshgrid(x, y)

        U = np.sin(np.load(measurment_path + '/' + aop_file_names_dict[time])[X, Y])
        V = np.cos(np.load(measurment_path + '/' + aop_file_names_dict[time])[X, Y])

        ax[3].quiver(Y, X, U, V, color = 'red', headlength=0, headaxislength=0, alpha = 1, linewidth = 4)
        

        ax[0].set_ylabel(f"Время {time.replace('_', ':')}", fontsize = 16)
    fig.tight_layout()
    plt.savefig(f"plots_exam/{ind}.png")
    plt.show()

In [ ]:
fig, ax = plt.subplots(1, 4, figsize = (20, 5), dpi = 200)

iteration = 100
time = "07_50_21"


for a in ax:
    a.set_xticks([])
    a.set_yticks([])
im = ax[0].imshow(np.load(measurment_path + '/' + dolp_file_names_dict[time]), cmap = 'rainbow')

plt.colorbar(im, fraction=0.046, pad=0.04)
im = ax[1].imshow(np.load(measurment_path + '/' + aop_file_names_dict[time]), cmap = 'rainbow')
plt.colorbar(im, fraction=0.046, pad=0.04)
ax[0].set_title("Степень поляризации", fontsize = 16)
ax[1].set_title("Угол поляризации", fontsize = 16)
ax[2].set_title("S0", fontsize = 16)
ax[3].set_title("Угол поляризации", fontsize = 16)

ax[2].imshow(np.load(measurment_path + '/' + s0_file_names_dict[time]), cmap = 'gray')


DOLP = np.load(measurment_path + '/' + dolp_file_names_dict[time])
min_polarization = np.unravel_index(DOLP.argmin(),
                                    DOLP.shape)


ax[3].imshow(np.load(measurment_path + '/' + s0_file_names_dict[time]), cmap = 'gray', alpha = 1)


x = np.arange(0, photo_x_shape, 30)
y = np.arange(0, photo_y_shape, 30)

ax[3].scatter([min_polarization[0]], [min_polarization[1]], label = 'Точка с минимальной поляризацией', color = 'blue', s = 40)
ax[2].scatter([min_polarization[0]], [min_polarization[1]], label = 'Точка с минимальной поляризацией', color = 'blue', s = 40)
ax[2].legend(loc = 'upper right')
X, Y = np.meshgrid(x, y)

U = np.sin(np.load(measurment_path + '/' + aop_file_names_dict[time])[X, Y])
V = np.cos(np.load(measurment_path + '/' + aop_file_names_dict[time])[X, Y])

ax[3].quiver(Y, X, U, V, color = 'red', headlength=0, headaxislength=0, alpha = 1, linewidth = 4)


ax[0].set_ylabel(f"Время {time.replace('_', ':')}", fontsize = 16)